# GET your API from Here 
- https://aistudio.google.com/app/apikey

In [35]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("Resume.pdf")
data = loader.load_and_split()
data

[Document(page_content='S. DINESH KUMAR || 2021PECAI250 \nsvani4830@gmail.com ||+91 9342372124 ||https://github.com/SDineshKumar1304 || www.linkedin.com/in/s-dineshkumar2004  \nPROFESSIONAL EXPERIENCE \nINNOMATICS RESEARCH LABS                                                                                                India, Hyderabad \nJunior Data Scientist Intern                                                                                             Jan 2024-Present (Remote) \nML Flow Model Experimentation: \nApplication Development: \nExploratory Data Analysis: \n● Developed ” Gen-AI A Multi-faceted AI Endeavor ”.Working on Gemini-1.5 pro  LLM  \n● Utilized Chroma DB  to develop a Search Engine  using tfidf-vectorizer and Cosine similarity  model for \nMovie Subtitles information retrieval system. \n● Developed Real Time Sentiment Analysis on Flipkart Products reviews web application with Flask \nDeployed the Web application on AWS \n● Utilized ML Flow and Prefect Orchestrati

In [36]:
from langchain_text_splitters import NLTKTextSplitter

text_splitter = NLTKTextSplitter(chunk_size=200, chunk_overlap=200)

chunks = text_splitter.split_documents(data)


Created a chunk of size 784, which is longer than the specified 200
Created a chunk of size 227, which is longer than the specified 200
Created a chunk of size 280, which is longer than the specified 200
Created a chunk of size 370, which is longer than the specified 200
Created a chunk of size 221, which is longer than the specified 200


In [40]:
for i in chunks:
    print(i)

page_content='S. DINESH KUMAR || 2021PECAI250 \nsvani4830@gmail.com ||+91 9342372124 ||https://github.com/SDineshKumar1304 || www.linkedin.com/in/s-dineshkumar2004  \nPROFESSIONAL EXPERIENCE \nINNOMATICS RESEARCH LABS                                                                                                India, Hyderabad \nJunior Data Scientist Intern                                                                                             Jan 2024-Present (Remote) \nML Flow Model Experimentation: \nApplication Development: \nExploratory Data Analysis: \n● Developed ” Gen-AI A Multi-faceted AI Endeavor ”.Working on Gemini-1.5 pro  LLM  \n● Utilized Chroma DB  to develop a Search Engine  using tfidf-vectorizer and Cosine similarity  model for \nMovie Subtitles information retrieval system.' metadata={'source': 'C:\\Users\\svani\\RAG\\Resume.pdf', 'page': 0}
page_content='● Developed Real Time Sentiment Analysis on Flipkart Products reviews web application with Flask \nDeployed 

In [38]:
print(len(chunks))

39


In [39]:
print(type(chunks[0]))

<class 'langchain_core.documents.base.Document'>


In [17]:

from langchain_community.vectorstores import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embedding_model = GoogleGenerativeAIEmbeddings(google_api_key="YOUR_API_KEY", 
                                               model="models/embedding-001")


In [18]:
# Embed each chunk and load it into the vector store
db = Chroma.from_documents(chunks, embedding_model, persist_directory="./chroma_db_")

# Persist the database on drive
db.persist()

C:\Users\svani\anaconda3\envs\tensorflow\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(


In [19]:
db_connection = Chroma(persist_directory="./chroma_db_", embedding_function=embedding_model)

In [20]:
retriever = db_connection.as_retriever(search_kwargs={"k": 1})

print(type(retriever))

<class 'langchain_core.vectorstores.VectorStoreRetriever'>


In [21]:
user_input = 'Explain this Resume'
retrieved_docs = retriever.invoke(user_input)

In [22]:
print(retrieved_docs[0].page_content)

ADDITIONAL INFORMATION 
● Technical Skills: Python,Flask,Streamlit,SQL&DBMS,EDA,MLOPS,Machine Learning,Data 
Analytics,PowerBI, NLP, AWS Deployment.


In [23]:
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
chat_template = ChatPromptTemplate.from_messages([
    # System Message Prompt Template
    SystemMessage(content="""You are a Helpful AI Bot. 
    You take the context and question from user. Your answer should be based on the specific context."""),
    # Human Message Prompt Template
    HumanMessagePromptTemplate.from_template("""Aswer the question based on the given context.
    Context:
    {context}
    
    Question: 
    {question}
    
    Answer: """)
])


In [24]:
from langchain_core.runnables import RunnablePassthrough

In [25]:

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [30]:
from langchain_google_genai import ChatGoogleGenerativeAI

chat_model = ChatGoogleGenerativeAI(google_api_key="YOUR_API_KEY", 
                                   model="gemini-1.5-pro-latest")

In [31]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [32]:

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | chat_template
    | chat_model
    | output_parser
)

In [33]:
response = rag_chain.invoke("Can you tell me about job?")

response

"## Understanding Your Potential Job Roles Based on Your Skills\n\nGiven your listed technical skills, you seem well-equipped for a variety of exciting roles in the data science and software development fields. Let's explore some potential job titles and what they might entail:\n\n**Data Science & Machine Learning:**\n\n* **Machine Learning Engineer:** You would be responsible for designing, building, and deploying machine learning models to solve real-world problems. Your skills in Python, ML, EDA, and MLOps would be crucial here.\n* **Data Scientist:** Analyzing large datasets, extracting insights, and communicating findings to stakeholders would be your core responsibility. Expertise in EDA, Machine Learning, PowerBI, and SQL would be essential.\n* **NLP Engineer:**  You would focus on developing systems that can understand and process human language. Your NLP and Python skills would be invaluable in this role.\n\n**Software Development & Deployment:**\n\n* **Backend Developer:** Bu

In [34]:
from IPython.display import Markdown as md

md(response)

## Understanding Your Potential Job Roles Based on Your Skills

Given your listed technical skills, you seem well-equipped for a variety of exciting roles in the data science and software development fields. Let's explore some potential job titles and what they might entail:

**Data Science & Machine Learning:**

* **Machine Learning Engineer:** You would be responsible for designing, building, and deploying machine learning models to solve real-world problems. Your skills in Python, ML, EDA, and MLOps would be crucial here.
* **Data Scientist:** Analyzing large datasets, extracting insights, and communicating findings to stakeholders would be your core responsibility. Expertise in EDA, Machine Learning, PowerBI, and SQL would be essential.
* **NLP Engineer:**  You would focus on developing systems that can understand and process human language. Your NLP and Python skills would be invaluable in this role.

**Software Development & Deployment:**

* **Backend Developer:** Building and maintaining the server-side logic of web applications would be your primary focus. Python, Flask, and SQL knowledge would be key.
* **Full-Stack Developer:** You would work on both the front-end and back-end of web applications, requiring proficiency in Python, Flask, and potentially JavaScript frameworks.
* **Cloud Engineer/DevOps Engineer:**  Your understanding of AWS deployment would be valuable for managing and deploying applications on cloud platforms.

**Data Analysis & Business Intelligence:**

* **Data Analyst:** You would gather, clean, and interpret data to answer business questions and support decision-making. Skills in SQL, PowerBI, and data analytics would be essential.
* **Business Intelligence Analyst:**  You would design and develop dashboards and reports to visualize data and track key performance indicators. Expertise in PowerBI and data analysis would be crucial.

**Additional Factors to Consider:**

* **Industry:** Your specific skills may be more relevant in certain industries than others. For example, NLP has applications in healthcare, finance, and customer service.
* **Experience Level:** Entry-level positions may focus on specific tasks, while senior roles often involve more leadership and strategic responsibilities.
* **Company Size and Culture:** Startups might offer more diverse responsibilities, while larger companies could provide structured career paths and mentorship.

**Exploring Job Opportunities:**

* **Job Boards:** Explore platforms like LinkedIn, Indeed, Glassdoor, and AngelList to discover relevant openings.
* **Networking:** Attend industry events, connect with professionals on LinkedIn, and reach out to your network for potential leads.
* **Company Websites:**  Check the career pages of companies you're interested in for open positions.

**Remember, this is not an exhaustive list, and your ideal job will depend on your interests, career goals, and experience.** 


In [ ]:
# END